# Chapter 4b - Plotting Tools

In this chapter we will learn to visualize data beyond a quick plot as in the previous chapter. We will present two examples (a time series plot and a map) using the libraries __matplotlib__ and __cartopy__. 

***

Let's start by importing the pertinent libraries.

In [ ]:
# basic libraries
import numpy as np
import pandas as pd
import xarray as xr

# necesary libraries for plotting
import matplotlib.pyplot as plt # note that in both cases we import one object within the library
import cartopy.crs as ccrs

import warnings
warnings.simplefilter('ignore') # filter some warning messages

## Plot SST anomaly timeseries

We will use the same data from the previous chapter to calculate and plot global sea surface temperature anomalies from the Hadley dataset. And we will also calculate the climatology and anomalies of monthly data to show a slightly more complicated plot, illustrating some of the __xarray__ methods. __Run the next cell.__

In [ ]:
# open the dataset
ds = xr.open_dataset('./data/HadISST_sst_2000-2020.nc') # read the netcdf file
ds.close() 

# select north and southern hemispheres, and average spatially to obtain a time series
nh_sst = ds.sst.sel(latitude=slice(90,0)).mean(dim=['latitude','longitude'])
sh_sst = ds.sst.sel(latitude=slice(0,-90)).mean(dim=['latitude','longitude'])

# calculate climatology
nh_clim = nh_sst.groupby('time.month').mean('time') # application of two methods:
# first groupby, and then the operation to perform over the group

# calculate and explore the anomalies
nh_ssta = nh_sst.groupby('time.month') - nh_clim  # groupby 'aligns' the data with the climatology, 
                                                # but only substract the appropiate climatology data point
nh_ssta # the new dataarray (one variable) has a new coordinate, but not dimension

### The actual plotting of the data
    
Making a simple plot using __matplotlib__ might seem like too much code, since there are many parameters to customize. However, it comes in handy for more detailed plots. __In the next cell__ we introduce some of the basic methods in a plot of the hemispheric averages calculated:
- Defining a figure and its size
- The function __plot__
- How to add labels and legend
- And how to display and 'finalize' a plot

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(nh_sst.time, nh_sst, '.-',label='NH') # the basic method plot() is used for line plots.
plt.plot(sh_sst.time, sh_sst, '+-', c='tab:orange', label='SH')
plt.grid(True)
plt.legend(loc=0)
plt.ylabel('SST (C)', fontsize=14)
plt.title('Monthly Hemisphheric SST', fontsize=16)
plt.show() # necesary line to finalize and properly display a figure

__In the next cell__ we plot the anomalies calculated, separating with color the positive and negative values. This is a more complicated plot that requires operating over the data first (using the method `.where`), but the plotting part is straight forward.

In [ ]:
plt.figure(figsize=(12,4))
pos = nh_ssta.where(nh_ssta>=0) # select only positive values 
neg = nh_ssta.where(nh_ssta<0)  # select only negative values
dates = nh_ssta.time.dt.year + (nh_ssta.time.dt.month-1)/12 # make a list of time steps using year and month
plt.bar(dates, pos.values, width=1/13, color='tab:red', edgecolor=None) # plot positive values
plt.bar(dates, neg.values, width=1/13, color='tab:blue') # plot negative values
plt.axhline(color='grey') # plot a grey horizontal line at y=0
plt.grid(True, zorder=0)
plt.ylabel('SST anomalies (C)')
plt.title('Northern Hemisphere SST anomalies')
plt.xticks([*range(2000,2021,1)], rotation=40)
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()


***
## Map plotting

Now we will customize our maps. While the quick plot method from __xarray__ is all we need in many cases, sometimes we require a more customized or nicer image for a presentation or publication. It might seem like complicated code, but really there are many elements that could be left to the default values, and we wanted to show how to customize some of them if you need them.

For global plots, the extent and the coordinate labels are sometimes not necessary to define, but we choose a regional plot for the next example to show how to customize these parameters. 

__Note__ that in the next to last line, we will also save our figure. We still use the _.show( )_ method in last line to display it.

In [ ]:
# import functions to label coordinates and add color to the land mass
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import calendar # quick access to names and numbers related to dates

# select a region of our data
region = np.array([[30,-40],[25,120]]) # numpy array that specifies the lat/lon boundaries of our selected region
io_sst = ds.sst.sel(latitude=slice(region[0,0],region[0,1]),longitude=slice(region[1,0],region[1,1])) # select region

for mon in [1,7]:  # select two months of data to plot: month 1 and month 7
    moname = calendar.month_name[mon] # get the name of the month
    tmp = io_sst.sel(time=ds.time.dt.month==mon).mean('time') # select only one month at a time in a temporal object

    # create and set the figure context
    fig = plt.figure(figsize=(8,5)) # create a figure object, and assign it a variable name fig
    ax = plt.axes(projection=ccrs.PlateCarree()) # projection type - this one is easy to use
    ax.coastlines(resolution='50m',linewidth=2,color='black') 
    ax.add_feature(cfeature.LAND, color='black')
    ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
    ax.set_xticks([*range(region[1,0],region[1,1]+1,20)], crs=ccrs.PlateCarree()) # customize ticks and labels to longitude
    ax.set_yticks([*range(region[0,1],region[0,0]+1,10)], crs=ccrs.PlateCarree()) # customize ticks and labels to latitude
    ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
    ax.yaxis.set_major_formatter(LatitudeFormatter())
    plt.grid(True, alpha=0.5) # add a grid. the alpha argument specify the level of transparency of a plot figure

    # the core: the data to plot
    plt.contourf(tmp.longitude,tmp.latitude, tmp,15, cmap='RdYlBu_r') # contourf (filled contour plot) takes the 1D lat and lon coordinates for the 2D data. cmap specify the colormap to use.
    cbar=plt.colorbar()
    cbar.set_label('SST (C)') # color bar label
    plt.title(moname+' SST (2000-2020)')
    fig.savefig('./figures/map_base_'+moname+'.png') # save your figure by usinig the method .savefig. python recognized the format from the filename extension. 
    plt.show()

***
### And that's it. Now you're ready to go over the application chapters and follow the code. Also you should be able to edit it and get your own results!

***
## Resources

[The Official __Matplotlib__ site](https://matplotlib.org/) 

Make sure to look at their [gallery](https://matplotlib.org/stable/gallery/index.html), which contains the code for each plot

A very simple, step by step [tutorial](https://github.com/rougier/matplotlib-tutorial) to matplotlib

[The Official __Cartopy__ -  site](https://scitools.org.uk/cartopy/docs/latest/), and [gallery](https://scitools.org.uk/cartopy/docs/latest/gallery/index.html)

R. Abernathey's [tutorial](https://rabernat.github.io/research_computing_2018/maps-with-cartopy.html) to Cartopy - Step by Step and very accessible

[__Seaborn__](https://seaborn.pydata.org/index.html) - We didn't talk about Seaborn, but it is a very nice library with beatiful and well designed functions for statistical data visualization. Make sure you take a look at their gallery

[The offical __Groupby__ reference](http://xarray.pydata.org/en/stable/groupby.html)

The __xarray__ page also have some useful examples for [weather](http://xarray.pydata.org/en/stable/examples/weather-data.html) and [climate](http://xarray.pydata.org/en/stable/examples/monthly-means.html) data that applies the methods (and more) used here.
